# DNA Sequence Classification by CNNs
Deep Learning Project. Dec 14, 2020
By Joely Nelson

In this project, I developed a convolutional neural network to classify DNA sequences from two-data sets. I mimic the architecture of the CNN used in prior work on two different datasets, and achieve close to their accuracy.

# Intro
DNA carries genetic instructions for the development, function, growth and reproduction of all known organisms. DNA is essentially the programming language of life; information stored as A, T, G, and C. It makes sense that we should be able to use DNA sequences to make predictions about a gene’s function, classification, and transcription levels. However, this is extremely difficult, as there are many unknown features, and many bioinformatics algorithms must be generated by hand.

With the emergence of deep learning, which makes use of automatic feature learning, and the increase of available genetic data sets, there is an opportunity to apply deep learning to genetic datasets to make complex predictions. Machine learning models and deep neural networks have been used to predict the non-coding function of DNA, identifying DNA-binding proteins, and even predicting the transcription level of genes. This is still a new field and there is much more to be explored.

In this project, I tried to reproduce some of the work done in the paper DNA Sequence Classification by Convolutional Neural Network. Researchers treated DNA data like text and used CNNs to classify various DNA sequences. By reproducing their architecture, I am able to fine tune models that achieve close to their accuracy or better.

I wanted to do this project because I am currently applying data-driven approaches to learn more about the rules of CRISPR in the Carothers Research Group at the university of Washington. I am planning on implementing a network to classify and generate sequences that are good for CRISPR. I would like to better my understanding of making models with sequence data so when data for my project is collected, I’m ready to hit the ground running.


# Setup

## Imports

Here are the packages we import to work on this project as well as any other general setup that needs to occur.

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import numpy as np
import os
import torch.nn.functional as F
import torch.optim as optim
import sys
import pandas as pd
import re
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
# from Bio import SeqIO 

from sklearn.model_selection import train_test_split

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## Preprocessing the data
DNA data is usually represented in strings made up of the characters A, T, G, and C. We can't train our network with just these strings and will need to do some kind of preprocessing of our data before we can feed it into our neural network.

<br>

### DNA As Strings
In the paper DNA Sequence Classification by Convolutional Neural Network, researchers treated DNA sequences like string tokens in language. Using this idea, we will attempt to convert DNA sequences into tokenized sequences.

We require a dictionary that maps the various tokens in the sequence (ie ATCG) to a numerical token. 

<br>

Here is an example of a mapping from sequence letters to integer tokens:

``{'A': 2, 'T': 3, 'C': 4, 'G':5}``

Using this map we can convert DNA sequences into tokenized sequences. For example this sequence:

``"GATCGTAGTCCTAGTAAGACTGAC"``

Would be tokenized as this:

``[5, 2, 3, 4, 5, 3, 2, 5, 3, 4, 4, 3, 2, 5, 3, 2, 2, 5, 2, 4, 3, 5, 2, 4]``

<br>

### Variable Sequence Lengths and Unknown Tokens
It is possible for some of the datasets that we will be analyzing, the various input sequences may be of different length. Unlike images or text, it ,may be unfeasible to resize or crop sequences. Instead we will add on an additional token, ``<pad>`` will go onto the tail end of any sequence that is too short.

In the future, someone else may want to use the dataset. It may also be possible that unknown tokens are found in that dataset, so we add on an ``<unk>`` token to represent unknown characters.

<br>

For example, if we had a dataset that consisted of the following two sequences:

'GATCGTAGTCCTAGTAAGACTGAC', 'TAGCC'

Then we might see a vocabulary to token mapping like this:

``{'<pad>': 0, '<unk>': 1, 'A': 2, 'T': 3, 'C': 4, 'G': 5}``

When tokenized and padded, the two sequences will look like this, respectivley. Note the long train of 0s on the smaller sequence to represent the padding.

``[5., 2., 3., 4., 5., 3., 2., 5., 3., 4., 4., 3., 2., 5., 3., 2., 2., 5., 2., 4., 3., 5., 2., 4.]``

``[3., 2., 5., 4., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]``


<br>
SOURCES:
We are going to use similiar thinking to sentence construction.
https://chriskhanhtran.github.io/posts/cnn-sentence-classification/

Data from: https://europepmc.org/article/med/16122420

In [ ]:
def label_to_count(labels):
    '''
    Given a list of labels, returns a dictionary that maps each class label to how many
    instances of that label were present in the list.
    '''
    label_to_count_dict = {}
    for label in labels:
        if label not in label_to_count_dict:
            label_to_count_dict[label] = 0
        label_to_count_dict[label] += 1
    return label_to_count_dict


def prepare_data(seqs):
    '''
    Given a list of sequences, will turn into a tokenized vector.
    
    ARGS:
        seqs: a list of strings where every string is a sequence
    RETURNS:
        tokenized_seqs (list(list(int))): list of list of tokens
        voc2ind (dict) a dictionary where keys are letters, values are the corresponding token
    '''
    max_len = 0
    
    # build up a voc2ind (letters:token)
    # based on ATGC and include padding and unknown tokens
    voc2ind = {voc:ind for ind,voc in enumerate(['<pad>', '<unk>', 'A', 'T', 'C', 'G'])}
    
    i = len(voc2ind)
    
    # tokenize the sequences
    tokenized_seqs = []
    for seq in seqs:
        tokenized_seq = []
        for e in seq:
            # make sure the sequence is upper case, a == A
            seq = seq.upper()
            # if we haven't seen this letter before, add to the corupus
            if not e in voc2ind:
                voc2ind[e] = i
                i += 1
            tokenized_seq.append(voc2ind[e])
        tokenized_seqs.append(tokenized_seq)
        
    return tokenized_seqs, voc2ind
        
res = prepare_data(['ATCG', 'TAGA', 'APO'])
print(res)
assert(res[0] == [[2, 3, 4, 5], [3, 2, 5, 2], [2, 6, 7]]), res[0]


def prepare_labels(labels):
    '''
    Given a list of labels will turn them into integer labels
    Args:
        labels: a list of labels
    Returns:
        tokenized_labels: numpy array(list) a list of label tokens
        label2token: (dict) a dictionary where keys are letters, values are corresponding token
    '''
    tokenized_labels = []
    label2token = {}
    i = 0
    for label in labels:
        if not label in label2token:
            label2token[label] = i
            i += 1
        tokenized_labels.append(label2token[label])
    return tokenized_labels, label2token


def pad(tokenized_seqs, voc2ind):
    '''
    Pad each sequence to the maximum length by adding a <pad> token
    
    ARGS:
        tokenized_seqs (list(list(str))): list of list of tokens
        voc2ind (dict) a dictionary where keys are letters, values are the corresponding token
    RETURNS:
        a numpy array of all the tokenized sequences that have been padded to be the same
        length.
    '''

    padded_seqs = []
    
    # find max sequence length
    max_len = 0
    for seq in tokenized_seqs:
        max_len = max(len(seq), max_len)
    
    # add padding so sequences are max_length
    for seq in tokenized_seqs:
        padded_seq = seq + [voc2ind['<pad>']] * (max_len - len(seq))
        padded_seqs.append(padded_seq)
        
    return np.array(padded_seqs, dtype=np.float32)


def data_loader(train_inputs, val_inputs, train_labels, val_labels,
                batch_size=50):
    """
    Convert train and validation sets to torch.Tensors and load them to
    DataLoader.
    """

    # Convert data type to torch.Tensor
    train_inputs, val_inputs, train_labels, val_labels =\
    tuple(torch.tensor(data) for data in
          [train_inputs, val_inputs, train_labels, val_labels])

    # Create DataLoader for training data
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

## Training
Below we have a generic training function which will train a given net. We also have an accuracy evaluation function for classification.

In [ ]:
def train(net, dataloader, epochs=1, lr=0.01, momentum=0.9, decay=0.0, verbose=1):
  ''' Trains a neural network. Returns a 2d numpy array, where every list 
  represents the losses per epoch.
  '''
  net.to(device)
  losses_per_epoch = []
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=decay)
  for epoch in range(epochs):
    sum_loss = 0.0
    losses = []
    for i, batch in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = batch[0].to(device), batch[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize 
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        losses.append(loss.item())
        sum_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            if verbose:
              print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, sum_loss / 100))
            sum_loss = 0.0
    # print(len(losses))
    losses_per_epoch.append(np.mean(losses))
  return losses_per_epoch

## Evaluation

Below are a few functions used for evaulating the function such as accuracy, printing the accuracy scores for a trained model, and plotting the loss over epochs.

In [ ]:
def accuracy(net, dataloader):
    '''
    Given a trained neural network and a dataloader, computes the accuracy.
    Arguments:
        net: a neural network
        dataloader: a dataloader
    Returns:
        fraction of examples classified correctly (float)
        number of correct examples (int)
        number of total examples (float)
    '''
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            input, labels = batch[0].to(device), batch[1].to(device)
            outputs = net(input)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct/total, correct, total

def print_eval(net, train_dataloader, test_dataloader):
    '''
    Given a test and train data loader, prints the test and train accuracy and
    the number of examples they got right.
    RETURNS
        (train_acc, test_acc) results of running accuracy on the two dataloaders
    '''
    train_acc = accuracy(net, train_dataloader)
    test_acc = accuracy(net, test_dataloader)
    

    print("Train accuracy: " + str(train_acc[0]) + "\t(" + str(train_acc[1]) + "/" + str(train_acc[2]) + ")")
    print("Test accuracy: " + str(test_acc[0]) + "\t(" + str(test_acc[1]) + "/" + str(test_acc[2]) + ")")
          
    return train_acc, test_acc

def plot_losses(losses, smooth_val = None, title = ""):
    '''
    Plots the losses per epoch returned by the training function.
    Args:
        losses: a list of losses returned by train
        smooth_val: an optinal integer value if smoothing is desired
        title: a title for the graph
    '''
    # loss = np.mean(losses, axis = 1)
    epochs = [i for i in range(1, len(losses) + 1)]
    if smooth_val is not None:
        lossses = smooth(losses, smooth_val)
    plt.plot(epochs, losses, marker="o", linestyle="dashed")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(title)
    

def smooth(x, size):
    '''
    Given an array, smooths it by some number size, to make it look less janky.
    '''
    return np.convolve(x, np.ones(size)/size, mode='same')



# Splice Dataset
https://archive.ics.uci.edu/ml/datasets/Molecular+Biology+(Splice-junction+Gene+Sequences)


This dataset contains information about splice junctions. In genes, regions which are removed during the RNA transcription process are called introns, and regions that are used to generate mRNA are called exons. Junctions between them are called splice junctions. There are two kinds of splice junction that is exon-intron junction, and intron-exon junctions.

Each of the sequences in this dataset are 60bp long, and belong to one of 3 classes: "EI" (Extron-Intron junction), "IE" (Intron-Extron junction) and "N" (neither EI or IE).

There are 767 genes with the EI label, 768 with the IE label, and 1655 with the N label.

In [ ]:
# loading dataset
# all sequences put in seq
# all labels in corresponding index in labels
seqs = []
labels = []

f = open('splice.data')
for line in f:
    line = [i.replace(',', '') for i in line.split()]
    seqs.append(line[2])
    labels.append(line[0])
f.close()

print(seqs)

# tokenizing and getting a vocab
tokenized_seqs, voc2ind = prepare_data(seqs)

# padding
tokenized_seqs = pad(tokenized_seqs, voc2ind)

# tokenizing labels
tokenized_labels, label2token = prepare_labels(labels)

# Showing the result of this:
print("\n", tokenized_seqs, "\n\n", voc2ind, "\n\n", label_to_count(labels))

# Train Test Split
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    tokenized_seqs, tokenized_labels, test_size=0.1, random_state=42)

# Load data to PyTorch DataLoader
train_dataloader, test_dataloader = data_loader(train_inputs, test_inputs, train_labels, test_labels, batch_size=50)

## Models

### Simple Model
This extremly simple neural network is to sanity check that the data has been processed correctly. Also if an extremly simple neural network with just a relu will classify output, that would also be great.

In [ ]:
FEATURE_SIZE = tokenized_seqs.shape[1]

class SimpleNet(nn.Module):
    def __init__(self, vocab_size, feature_size, num_classes=3):
        super(SimpleNet, self).__init__()
        # UM?
        self.vocab_size = vocab_size
        # embeded_dim: Dimension of word vectors.
        self.feature_size = feature_size

        self.encoder = nn.Embedding(self.vocab_size, self.feature_size)

        self.fc1 = nn.Linear(feature_size, 1000)
        self.fc2 = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# Creating net and training
net = SimpleNet(len(voc2ind), FEATURE_SIZE)
losses = train(net, train_dataloader, epochs = 10)

# Evaluation
plot_losses(losses, title = "Splice Dataset: Convolutional Network Loss")
print_eval(net, train_dataloader, test_dataloader)
print()

This model seems to classify quite alright, even with such a simple network.

### Convolutional Model with 2D Max Pooling
We will now attempt to implement the model from the [DNA Sequence Classification by Convolutional Neural Network paper](https://www.researchgate.net/publication/301703031_DNA_Sequence_Classification_by_Convolutional_Neural_Network) Here is the discussion that they leave:

"*The model contains 2 convolutional layers. Each of these layers is followed by a sub-sampling layer. These layers are used to extract features from representation matrixes  of  sequences.  The  extracted  features  are  then  transformed  by  using  a  fully  connected  neural  network  layer which contains 100 neurons. In this layer, we used a dropout value of 0.5 to reduce the effect overfitting. Finally, a soft max output layer is used to predict labels of input sequences.*"

This is not a lot to go off of. The researchers do not mention an activation function, but we will use one because otherwise this model is no better than a linear one. We will attempt to follow their architecture as closely as possible and attempt to get their accuracy score of 96%.

In [ ]:
480/60 * 120

In [ ]:

# Here are some experiments run with the model
# kind of represents a table, each row is an experiment.
# the first row is the indecies. Each row should be the same length.
# test acc and train acc recorded for 10 epochs, are not neccesarily the final model accuracy reported in my report
ex = [['in_channels_conv1', 'out_channels_conv1', 'in_channels_conv2', 'out_channels_conv2', 'in_channels_fc1', 'kernel_size', 'stride', ('train_acc'), ('test_acc')],
      [60, 40, 20, 40, 40, 3, 3], #1
      [60, 240, 120, 480, 480, 3, 3], #2 
      [60, 480, 240, 960, 960, 3, 3, 0, (0.973528), (0.918495)], #3
      [60, 480, 240, 960, 960, 6, 3, 0, (0.998607), (0.962382)], #4
      [60, 480, 240, 960, 960, 6, 3,  0.01, (0.991988), (0.9717)], #5 BEST ONE! :)
      [10, 80, 40, 160, 160, 6, 3,  0.01, (0.9738766980146291), (0.9561128526645768)], #6
      [60, 480, 240, 960, 960, 6, 3,  0.02, (0.9641239986067572), (0.934169278996865)], #7
]
i = 5

FEATURE_SIZE = ex[i][0]

class DSC_by_CNN(nn.Module):
    def __init__(self, vocab_size, feature_size, num_classes=3):


        super(DSC_by_CNN, self).__init__()
        # UM?
        self.vocab_size = vocab_size
        # embeded_dim: Dimension of word vectors.
        self.feature_size = feature_size

        self.encoder = nn.Embedding(self.vocab_size, self.feature_size)
        

        self.conv1 = nn.Conv1d(in_channels=ex[i][0],
                               out_channels=ex[i][1], 
                               kernel_size=ex[i][5],
                               stride = ex[i][6],
                               padding = 1)
        self.conv2 = nn.Conv1d(in_channels=ex[i][2],
                               out_channels=ex[i][3], 
                               kernel_size=ex[i][5],
                               stride = ex[i][6],
                               padding = 1)


        # Activation
        self.act = nn.ReLU()

        # Pooling
        self.pool = nn.MaxPool2d(3, 2, 1)

        # fully connected layer
        self.fc1 = nn.Linear(ex[i][4], 100)

        # dropout
        self.drop = nn.Dropout(p=0.5)


    # seen in the network pytorch tutorial
    # I've been using this function for years, no idea what it does.
    # Would it kill the pytorch tutorial people to document this guy?
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features        


    def forward(self, x):
        # The following line is required to send it through the encoder. Not sure why.
        # b_input_ids = torch.tensor(b_input_ids).to(device).long()
        x = torch.tensor(x).to(torch.int64)

        # Encode. 
        # Output shape: (b, max_len, embedded_dim)
        x = self.encoder(x)

        # Permute shuffles dimensions. Do this to satisfy requirments of nn.Conv1d
        # Output shape: (b, embedded_dim, max_len)
        x = x.permute(0, 2, 1)

        # convolutional layer 1
        x = self.conv1(x)
        x = self.act(x)
        x = self.pool(x)

        # convolutional layer 2
        x = self.conv2(x)
        x = self.act(x)
        x = self.pool(x)

        # flattening
        x = x.view(-1, self.num_flat_features(x))

        # Tring to figure out the shape
        #print(x.shape)

        # fully linear layer + dropout
        x = self.drop(x)
        # x = self.softm(x)
        x = self.fc1(x)

        return x

# Creating net and training
net = DSC_by_CNN(len(voc2ind), FEATURE_SIZE)
losses = train(net, train_dataloader, epochs=50, lr=0.01, decay = ex[i][-3])

# Evaluation
plot_losses(losses, title = "Splice Dataset: Convolutional Network Loss")
print_eval(net, train_dataloader, test_dataloader)
print()

### Convolutional Model with 1D Max Pooling

In [ ]:

# Here are some experiments run with the model
# kind of represents a table, each row is an experiment.
# the first row is the indecies. Each row should be the same length.
ex = [['in_channels_conv1', 'out_channels_conv1', 'in_channels_conv2', 'out_channels_conv2', 'in_channels_fc1', 
       'kernel_size_conv1', 'stride_conv1', 'kenrnel_size_conv2', 'kernel_size_conv1', 
       'momentum', 'learning rate', 'decay', ('train_acc'), ('test_acc'),],
      [60, 480, 480, 960, 1920, 6, 3, 6, 3,   0.8,0.01, 0.02, (0.8193495693495694), (0.7989311957247829)], #1
]
i = 1

FEATURE_SIZE = ex[i][0]

class H3_DSC_by_CNN(nn.Module):
    def __init__(self, vocab_size, feature_size, num_classes=2):


        super(H3_DSC_by_CNN, self).__init__()
        # UM?
        self.vocab_size = vocab_size
        # embeded_dim: Dimension of word vectors.
        self.feature_size = feature_size

        self.encoder = nn.Embedding(self.vocab_size, self.feature_size)
        

        self.conv1 = nn.Conv1d(in_channels=ex[i][0],
                               out_channels=ex[i][1], 
                               kernel_size=ex[i][5],
                               stride = ex[i][6],
                               padding = 1)
        self.conv2 = nn.Conv1d(in_channels=ex[i][2],
                               out_channels=ex[i][3], 
                               kernel_size=ex[i][7],
                               stride = ex[i][8],
                               padding = 1)


        # Activation
        self.act = nn.ReLU()

        # Pooling
        self.pool = nn.MaxPool1d(3, 2, 1)

        # fully connected layer
        self.fc1 = nn.Linear(ex[i][4], 100)

        # dropout
        self.drop = nn.Dropout(p=0.5)

        # softmax
        self.softm = nn.Softmax()


    # seen in the network pytorch tutorial
    # I've been using this function for years, no idea what it does.
    # Would it kill the pytorch tutorial people to document this guy?
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features        


    def forward(self, x):
        # The following line is required to send it through the encoder. Not sure why.
        # b_input_ids = torch.tensor(b_input_ids).to(device).long()
        x = torch.tensor(x).to(torch.int64)

        # Encode. 
        # Output shape: (b, max_len, embedded_dim)
        x = self.encoder(x)

        # idk flatten it?
        # x = torch.flatten(x, 1)

        # Permute shuffles dimensions. Do this to satisfy requirments of nn.Conv1d
        # Output shape: (b, embedded_dim, max_len)
        x = x.permute(0, 2, 1)

        # convolutional layer 1
        x = self.conv1(x)
        x = self.act(x)
        x = self.pool(x)

        # convolutional layer 2
        x = self.conv2(x)
        x = self.act(x)
        x = self.pool(x)

        # flattening?
        x = x.view(-1, self.num_flat_features(x))

        # Tring to figure out the shape
        # print(x.shape)

        # fully linear layer + dropout
        x = self.drop(x)
        # x = self.softm(x)
        x = self.fc1(x)

        return x

# Creating net and training
net = H3_DSC_by_CNN(len(voc2ind), FEATURE_SIZE)
losses = train(net, train_dataloader, epochs = 20, decay = ex[i][-3], lr=ex[i][-4], momentum=ex[i][-5])

# Evaluation
plot_losses(losses, title = "Splice Dataset: Convolutional Network Loss")
print_eval(net, train_dataloader, test_dataloader)
print()

# H3 Dataset
http://www.jaist.ac.jp/~tran/nucleosome/members.htm 

These datasets are about DNA sequences wrapping around histone proteins. It is  mechanism used to pack and store long DNA sequence into a cell’s nucleus. Name convention in these datasets is constructed as follows: “H3” or “H4” indicates the histone type, “K” and a succeeding number indicate a modified amino acid (e.g. “K14” denotes the 14th amino acid “K” has been modified), and “ac” or “me” indicate the type of modification (acetylation or methylation) and a num-
ber after “me” indicates times of methylation. In each dataset, samples are sequences with length of 500 base pairs and belong to “Positive” or “Negative” class. Samples in “Positive” class contain regions wrapping around histone proteins. In contrast, samples in “Negative” class do not contain them. With these datasets, if we could predict histone profiles from sequences with a certain level of accuracy we might help to understand about ex-
pression pattern of genes. 


In [ ]:
seqs = []
labels = []

# every sequence comes as 3 lines. The first line is the name,
# the second is the gene
# the third is the class

f = open('h3.txt')
for i, line in enumerate(f):
    if i % 3 == 1:
        line = re.sub('[^ATCGatcg]', '', line)
        seqs.append(line)
    if i%3 == 2:
        labels.append(int(line[0]))
f.close()

print(seqs[:5])
print(labels[:5])

# tokenizing and getting a vocab
tokenized_seqs, voc2ind = prepare_data(seqs)

# padding
tokenized_seqs = pad(tokenized_seqs, voc2ind)

# tokenizing labels
tokenized_labels, label2token = prepare_labels(labels)

# Showing the result of this:
print("\n", tokenized_seqs, "\n\n", voc2ind, "\n\n", label_to_count(labels))

# Train Test Split
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    tokenized_seqs, tokenized_labels, test_size=0.1, random_state=42)

# Load data to PyTorch DataLoader
train_dataloader, test_dataloader = data_loader(train_inputs, test_inputs, train_labels, test_labels, batch_size=50)

## Models

### Simple Model

This extremly simple neural network is to sanity check that the data has been processed correctly. Also if an extremly simple neural network with just a relu will classify output, that would also be great.

In [ ]:
FEATURE_SIZE = tokenized_seqs.shape[1]

class SimpleNet(nn.Module):
    def __init__(self, vocab_size, feature_size, num_classes=3):
        super(SimpleNet, self).__init__()
        # UM?
        self.vocab_size = vocab_size
        # embeded_dim: Dimension of word vectors.
        self.feature_size = feature_size

        self.encoder = nn.Embedding(self.vocab_size, self.feature_size)

        self.fc1 = nn.Linear(feature_size, 1000)
        self.fc2 = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# Creating net and training
net = SimpleNet(len(voc2ind), FEATURE_SIZE)
losses = train(net, train_dataloader, epochs = 10)

# Evaluation
plot_losses(losses, title = "H3: Simple Network Loss")
print_eval(net, train_dataloader, test_dataloader)
print()

### Convolutional Model with 2D Max Pooling
Goal to beat: 88.99%

This number is not beat.


In [ ]:
# Here are some experiments run with the model
# kind of represents a table, each row is an experiment.
# the first row is the indecies. Each row should be the same length.
ex = [['in_channels_conv1', 'out_channels_conv1', 'in_channels_conv2', 'out_channels_conv2', 'in_channels_fc1', 
       'kernel_size_conv1', 'stride_conv1', 'kenrnel_size_conv2', 'kernel_size_conv1', 
       'momentum', 'learning rate', 'decay', ('train_acc'), ('test_acc'),],
      [500, 4000, 2000, 8000, 56000, 6, 3, 6, 3,    0.9, 0.01, 0, ("NaN"), ("NaN")], #1
      [500, 1000, 500, 1000, 7000, 6, 3, 6, 3,     0.9,0.01, 0, (0.752895752895753), (0.7441549766199065), 0], #2
      [500, 2000, 1000, 4000, 28000, 6, 3, 6, 3,     0.9,0.01, 0, (0.7117612117612118), (0.6913827655310621)], #3
      [500, 1000, 500, 1000, 7000, 3, 3, 3, 3,     0.9,0.01, 0, (0.6470151470151471), (0.6479625918503674)], #4
      [500, 1000, 500, 1000, 1500, 12, 6, 12, 6,     0.9,0.01, 0, (0.7730917730917731), (0.7234468937875751)], #5
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.01, 0, (0.8271458271458272), (0.7682030728122913)], #6
      [500, 1500, 750, 1500, 5250, 12, 6, 6, 3,     0.9,0.01, 0, (0.7684882684882685), (0.6947227788911156)], #7
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.01, 0.01, (0.8708048708048708), (0.8162992651970608)], #8
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.01, 0.001, (0.7771755271755272), (0.7207748830995324)], #9
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.01, 0.1, (0.7817047817047817), (0.7762191048764195)], #10
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.01, 0.025, (0.8084348084348084), (0.7989311957247829)], #11
      [500, 750, 375, 500, 1750, 12, 6, 6, 3,     0.9,0.01, 0.025, (0.7955895455895456), (0.7802271209084837)], #12
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.01, 0.0175, (0.7465102465102466), (0.730126920507682)], #13
      [500, 1500, 750, 1500, 5250, 12, 6, 6, 3,     0.9,0.01, 0.025, (0.7681170181170182), (0.7615230460921844)], #14
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.001, 0.025, (0.8938223938223938), (0.822311289245157)], #15: (This delivers the best test accuracy)
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.001, 0.05, (0.7963320463320464), (0.781563126252505)], #16
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.001, 0.035, (0.8566231066231066), (0.7975951903807615)], #17
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.0001, 0.025, (0.8264775764775765), (0.7909151636606546)], #18 #
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.001, 0.025, (0.8693198693198693), (0.8089512358049432)], #19
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.75,0.001, 0.025, (0.8855806355806356), (0.8203072812291249)], #2
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.85,0.001, 0.03, (0.876002376002376), (0.8183032732130928)], #21
      [500, 1250, 625, 1400, 4900, 12, 6, 6, 3,     0.85,0.001, 0.025, (0.8944163944163944), (0.8156312625250501)], #22
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.85, 0.001,0.025, (0.888921888921889), (0.8236472945891784)], #23
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.85,0.001, 0.03, (0.903920403920404), (0.8203072812291249)], #24
      [60, 480, 240, 960, 6720, 6, 3, 6, 3,     0.9,0.01, 0, (0.962949212949213), (0.7748830995323981)], #25
      [60, 480, 240, 960, 6720, 6, 3, 6, 3,     0.9,0.01, 0.0025, (0.924933174933175), (0.7849031396125584)], #26
      [100, 800, 400, 1600, 11200, 6, 3, 6, 3,     0.9,0.01, 0.0025, (0.8551381051381052), (0.7047428189712759)], #27
      [60, 240, 120, 240, 1680, 6, 3, 6, 3,     0.9,0.01, 0, (0.8951588951588951), (0.7702070808283233)], #28
      [60, 480, 240, 960, 3360, 12, 6, 6, 3,     0.9,0.01, 0, (0.962949212949213), (0.7748830995323981)], #29
      [60, 480, 240, 960, 3360, 12, 6, 6, 3,     0.9,0.01, 0.03, (0.8003415503415503), (0.7895791583166333)], #30
      [60, 480, 240, 960, 3360, 12, 6, 6, 3,     0.9,0.01, 0.01, (), ()], #31
      [60, 480, 240, 960, 3360, 12, 6, 6, 3,     0.9,0.01, 0.02, (0.8152658152658153), (0.800935203740815)], #32
      [60, 480, 240, 960, 3360, 12, 6, 6, 3,     0.8,0.01, 0.02, (0.8415503415503416), (0.8096192384769539)], #33
      [60, 480, 240, 960, 60000, 3, 1, 3, 1,     0.9,0.01, 0.02, (0.7912087912087912), (0.7688710754843019)], #34
      [60, 480, 240, 960, 20160, 6, 3, 3, 1,     0.9,0.01, 0.02, (0.7833382833382834), (0.7561790247160989)], #35
      [60, 480, 240, 960, 14880, 6, 2, 6, 2,     0.9,0.01, 0.02, (0.796035046035046), (0.7855711422845691)], #36
      [60, 480, 240, 960, 14880, 6, 2, 6, 2,     0.9,0.01, 0.0, (0.5092070092070092), (0.5156980627922512)], #37
      [60, 480, 240, 960, 14880, 6, 2, 6, 2,     0.9,0.01, 0.01, (0.7923225423225423), (0.7762191048764195)], #38
      [250, 500, 250, 500, 1750, 12, 6, 6, 3,     0.9,0.01, 0.01, (0.7711612711612712), (0.7561790247160989)], #39
      [25, 500, 250, 500, 1750, 12, 6, 6, 3,     0.9,0.01, 0.01, (0.7711612711612712), (0.7561790247160989)], #40
      [500, 1000, 500, 1000, 3500, 12, 6, 6, 3,     0.9,0.001, 0.03, (0.8853578853578854), (0.81)], #41
      [60, 480, 240, 960, 60000, 4, 1, 4, 1,     0.9,0.01, 0.02, (0.805019305019305), (0.7882431529726119)], #42
      [25, 100, 50, 200, 12500, 4, 1, 4, 1,     0.9,0.01, 0.02, (0.7852), (0.7649)], #43
      [25, 100, 50, 200, 12500, 4, 1, 4, 1,     0.9,0.01, 0.00, (0.87), (0.77)], #44  
      [50, 100, 50, 200, 12500, 4, 1, 4, 1,     0.9,0.01, 0.02, (0.69), (0.69)], #45
      [500, 1000, 500, 1000, 62500, 4, 1, 4, 1,     0.9,0.001, 0.025, (0.8666), (0.8029)], #46
      [60, 480, 240, 960, 60000, 4, 1, 4, 1,     0.9,0.001, 0.025, (0.752), (0.73)], #47
      [4, 100, 50, 200, 12500, 4, 1, 4, 1,     0.9,0.001, 0.025, (0.822), (0.8036)], #48
      [4, 100, 50, 200, 12500, 4, 1, 4, 1,     0.9,0.001, 0.0, (0.822), (0.8036)], #49
      [1, 50, 25, 200, 1400, 6, 3, 6, 3,     0.9,0.001, 0.025, (0.8094), (0.7922)], #50
      [2, 100, 50, 400, 2800, 6, 3, 6, 3,     0.9,0.001, 0.025, (0.8094), (0.7922)], #51
      [2, 100, 50, 400, 2800, 6, 3, 6, 3,     0.9,0.001, 0.005, (0.8587), (0.8243)], #52 (BEST)
      [2, 150, 75, 400, 2800, 6, 3, 6, 3,     0.9,0.001, 0.01, (0.8094), (0.7922)], #53
]
i = 52

FEATURE_SIZE = ex[i][0]

class H3_DSC_by_CNN(nn.Module):
    def __init__(self, vocab_size, feature_size, num_classes=2):


        super(H3_DSC_by_CNN, self).__init__()
        # UM?
        self.vocab_size = vocab_size
        # embeded_dim: Dimension of word vectors.
        self.feature_size = feature_size

        self.encoder = nn.Embedding(self.vocab_size, self.feature_size)
        

        self.conv1 = nn.Conv1d(in_channels=ex[i][0],
                               out_channels=ex[i][1], 
                               kernel_size=ex[i][5],
                               stride = ex[i][6],
                               padding = 1)
        self.conv2 = nn.Conv1d(in_channels=ex[i][2],
                               out_channels=ex[i][3], 
                               kernel_size=ex[i][7],
                               stride = ex[i][8],
                               padding = 1)


        # Activation
        self.act = nn.ReLU()

        # Pooling
        self.pool = nn.MaxPool2d(3, 2, 1)

        # fully connected layer
        self.fc1 = nn.Linear(ex[i][4], 100)
        self.fc2 = nn.Linear(100, num_classes)

        # dropout
        self.drop = nn.Dropout(p=0.5)

        # softmax
        self.softm = nn.Softmax()


    # seen in the network pytorch tutorial
    # I've been using this function for years, no idea what it does.
    # Would it kill the pytorch tutorial people to document this guy?
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features        


    def forward(self, x):
        # The following line is required to send it through the encoder. Not sure why.
        # b_input_ids = torch.tensor(b_input_ids).to(device).long()
        x = torch.tensor(x).to(torch.int64)

        # Encode. 
        # Output shape: (b, max_len, embedded_dim)
        x = self.encoder(x)

        # idk flatten it?
        # x = torch.flatten(x, 1)

        # Permute shuffles dimensions. Do this to satisfy requirments of nn.Conv1d
        # Output shape: (b, embedded_dim, max_len)
        x = x.permute(0, 2, 1)

        # convolutional layer 1
        x = self.conv1(x)
        x = self.act(x)
        x = self.pool(x)

        # convolutional layer 2
        x = self.conv2(x)
        x = self.act(x)
        x = self.pool(x)

        # flattening?
        x = x.view(-1, self.num_flat_features(x))

        # Tring to figure out the shape
        # print(x.shape)

        # fully linear layer + dropout
        x = self.drop(x)
        # x = self.softm(x)
        x = self.fc1(x)
        # x = self.fc2(x)

        return x

# Creating net and training
net = H3_DSC_by_CNN(len(voc2ind), FEATURE_SIZE)
losses = train(net, train_dataloader, epochs = 150, decay = ex[i][-3], lr=ex[i][-4], momentum=ex[i][-5])

# Evaluation
plot_losses(losses, title = "H3 Dataset: Convolutional Network Loss")
print_eval(net, train_dataloader, test_dataloader)
print()

### Convolutional Model with 1D Max Pooling

Goal to beat: 88.99%

This number is not beat.


In [ ]:

# Here are some experiments run with the model
# kind of represents a table, each row is an experiment.
# the first row is the indecies. Each row should be the same length.
ex = [['in_channels_conv1', 'out_channels_conv1', 'in_channels_conv2', 'out_channels_conv2', 'in_channels_fc1', 
       'kernel_size_conv1', 'stride_conv1', 'kenrnel_size_conv2', 'kernel_size_conv1', 
       'momentum', 'learning rate', 'decay', ('train_acc'), ('test_acc'),],
      [60, 480, 480, 960, 6720, 12, 6, 6, 3,     0.8,0.01, 0.02, (0.8193495693495694), (0.7989311957247829)], #1
      [60, 240, 240, 960, 6720, 12, 6, 6, 3,     0.8,0.01, 0.02, (0.788016038016038), (0.7768871075484302)], #2
      [60, 600, 600, 960, 6720, 12, 6, 6, 3,     0.8,0.01, 0.02, (0.8154885654885655), (0.7935871743486974)], #3
      [60, 480, 480, 960, 6720, 12, 6, 6, 3,     0.9,0.01, 0.0, (0.9832194832194833), (0.772879091516366)], #4
      [60, 480, 480, 960, 6720, 12, 6, 6, 3,     0.9,0.01, 0.001, (0.9861152361152361), (0.7828991315965264)], #5
      [60, 600, 600, 960, 6720, 12, 6, 6, 3,     0.8,0.01, 0.01, (0.8964211464211465), (0.8169672678690715)], #6
      [60, 600, 600, 960, 13440, 6, 3, 6, 3,     0.8,0.01, 0.02, (0.809028809028809), (0.7822311289245157)], #7
      [60, 960, 960, 1920, 13440, 12, 6, 6, 3,     0.8,0.01, 0.02, (0.8181615681615682), (0.7989311957247829)], #8
      [5, 40, 40, 80, 560, 12, 6, 6, 3,     0.9,0.01, 0.0, (0.8316750816750816), (0.7742150968603875)], #9
      [10, 80, 80, 160, 1120, 12, 6, 6, 3,     0.9,0.01, 0.0, (0.9221116721116721), (0.7775551102204409)], #10
      [10, 80, 80, 160, 1120, 12, 6, 6, 3,     0.9,0.01, 0.001, (0.9102316602316602), (0.7862391449565799)], #11
      [5, 40, 40, 80, 1120, 6, 3, 6, 3,     0.9,0.01, 0.0, (0.8594446094446094), (0.8002672010688042)], #12
      [5, 20, 20, 40, 560, 6, 3, 6, 3,     0.9,0.01, 0.0, (0.7884615384615384), (0.7535070140280561)], #13
      [5, 40, 40, 80, 1120, 6, 3, 6, 3,     0.85,0.01, 0.001, (0.8424413424413424), (0.7929191716766867)], #14
      [5, 50, 50, 100, 1400, 6, 3, 6, 3,     0.9,0.01, 0.0, (0.8059103059103059), (0.7528390113560455)], #15
      [5, 30, 30, 60, 840, 6, 3, 6, 3,     0.9,0.01, 0.0, (0.8176418176418176), (0.7595190380761523)], #16
      [6, 40, 40, 80, 1120, 6, 3, 6, 3,     0.9,0.01, 0.02, (0.8594446094446094), (0.8002672010688042)], #17
      [2, 100, 100, 400, 5600, 6, 3, 6, 3,     0.9,0.001, 0.005, (0.8587), (0.8243)], #18
]
i = 18

FEATURE_SIZE = ex[i][0]

class H3_DSC_by_CNN(nn.Module):
    def __init__(self, vocab_size, feature_size, num_classes=2):


        super(H3_DSC_by_CNN, self).__init__()
        # UM?
        self.vocab_size = vocab_size
        # embeded_dim: Dimension of word vectors.
        self.feature_size = feature_size

        self.encoder = nn.Embedding(self.vocab_size, self.feature_size)
        

        self.conv1 = nn.Conv1d(in_channels=ex[i][0],
                               out_channels=ex[i][1], 
                               kernel_size=ex[i][5],
                               stride = ex[i][6],
                               padding = 1)
        self.conv2 = nn.Conv1d(in_channels=ex[i][2],
                               out_channels=ex[i][3], 
                               kernel_size=ex[i][7],
                               stride = ex[i][8],
                               padding = 1)


        # Activation
        self.act = nn.ReLU()

        # Pooling
        self.pool = nn.MaxPool1d(3, 2, 1)

        # fully connected layer
        self.fc1 = nn.Linear(ex[i][4], 100)

        # dropout
        self.drop = nn.Dropout(p=0.5)

        # softmax
        self.softm = nn.Softmax()


    # seen in the network pytorch tutorial
    # I've been using this function for years, no idea what it does.
    # Would it kill the pytorch tutorial people to document this guy?
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features        


    def forward(self, x):
        # The following line is required to send it through the encoder. Not sure why.
        # b_input_ids = torch.tensor(b_input_ids).to(device).long()
        x = torch.tensor(x).to(torch.int64)

        # Encode. 
        # Output shape: (b, max_len, embedded_dim)
        x = self.encoder(x)

        # idk flatten it?
        # x = torch.flatten(x, 1)

        # Permute shuffles dimensions. Do this to satisfy requirments of nn.Conv1d
        # Output shape: (b, embedded_dim, max_len)
        x = x.permute(0, 2, 1)

        # convolutional layer 1
        x = self.conv1(x)
        x = self.act(x)
        x = self.pool(x)

        # convolutional layer 2
        x = self.conv2(x)
        x = self.act(x)
        x = self.pool(x)

        # flattening?
        x = x.view(-1, self.num_flat_features(x))

        # Tring to figure out the shape
        # print(x.shape)

        # fully linear layer + dropout
        x = self.drop(x)
        # x = self.softm(x)
        x = self.fc1(x)

        return x

# Creating net and training
net = H3_DSC_by_CNN(len(voc2ind), FEATURE_SIZE)
losses = train(net, train_dataloader, epochs = 50, decay = ex[i][-3], lr=ex[i][-4], momentum=ex[i][-5])

# Evaluation
plot_losses(losses, title = "H3 Dataset: Convolutional Network Loss")
print_eval(net, train_dataloader, test_dataloader)
print()